# Factorization Machine
In this article, we’ll introduce Factorization Machines (FM) as a flexible and powerful modeling framework for collaborative filtering recommendation. <br>
Specifically, we will demonstrate how FM achieves second-order interaction which improves the performance on top of the first-order model.<br>


FM is an effective method to handle data with high dimensionality categorical data like the following figure.
![](https://miro.medium.com/max/1400/1*MKqlUaZGiECvqTOJElmfhg.png)

In [ ]:
# install data
! wget https://zenodo.org/record/5700987/files/Criteo_x1.zip
! unzip Criteo_x1.zip -d Criteo

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader,Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
# data = pd.read_csv('./criteo_sample.txt')
data = pd.read_csv('./Criteo/train.csv',nrows=1e5)

# separate numerical and categorical features
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data.head(3)

Let's take a look at the number of unique categorical variables in each column

In [ ]:
# preprocess
data[sparse_features] = data[sparse_features].fillna('-1', )
data = data.drop(dense_features,axis=1)
data = data.apply(lambda x:LabelEncoder().fit_transform(x))
targets = data['label']
data = data.drop("label",axis=1)

# calculate field dimensions
field_dims = data.nunique().values
print(field_dims)

In [ ]:
class CriteoDataset(Dataset):
    def __init__(self, num_fields, features, target):
        super().__init__()
        self.num_fields = num_fields
        self.features = torch.LongTensor(features)
        self.target = torch.FloatTensor(target)
        
    def __getitem__(self,index):
        return self.features[index,:], self.target[index]
    
    def __len__(self):
        return len(self.features)

In [ ]:
dataset = CriteoDataset(field_dims, data.values, targets.values)

# split dataset
train_length = int(len(dataset) * 0.8)
valid_length = int(len(dataset) * 0.1)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
        dataset, (train_length, valid_length, test_length))

# loader
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=2,shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

# From first-order to second-order interaction

## First-order model: Logistic regression
Normally, when we think of linear regression, we would think of the following formula:
$$
\hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i
$$
Where:
$\mathbf{w}_0$  is the bias term, a.k.a intercept.
$\mathbf{w}_i$  are weights corresponding to each feature vector $x_i$ , here we assume we have  $n$  total features.
This formula's advantage is that it can computed in linear time, $𝑂(𝑛)$ . The drawback, however, is that it does not handle feature interactions. <br>
Let's first implement the LR model and see how it works!

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, field_dims, output_dim=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        # Since the value of each dimension starts with 0
        # we need to add the offsets of previous dimensions
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        
        # first-order term (i.e. \sum w_i x_i)
        first_order = torch.sum(self.fc(x), dim=1)
        
        # zero-order (i.e. bias)
        zero_order = self.bias
        
        output = first_order + zero_order
        
        return output.flatten()

In [ ]:
from sklearn.metrics import roc_auc_score
def test(model, data_loader, device):
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in data_loader:
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return roc_auc_score(targets, predicts)

In [ ]:
# training configs
EPOCHS = 10
lr = 1e-4
device = "cuda"

# build model
model = LogisticRegression(field_dims).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
objective_function = nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(EPOCHS):
    for fields, target in train_loader:
        fields, target = fields.to(device), target.to(device)
        prediction = model(fields)
        loss = objective_function(prediction, target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    valid_roc = test(model, valid_loader,device)
    print(f"Validation ROC score:{valid_roc:.4f}")
    
test_roc = test(model, test_loader,device)
print(f"Testing ROC score:{test_roc:.4f}")

# Factorization Machines

Factorization machines (FM) `Rendle.2010`, proposed by Steffen Rendle in 2010, is a supervised algorithm that can be used for classification, regression, and ranking tasks. It quickly took notice and became a popular and impactful method for making predictions and recommendations. Particularly, it is a generalization of the linear regression model and the matrix factorization model. Moreover, it is reminiscent of support vector machines with a polynomial kernel. The strengths of factorization machines over the linear regression and matrix factorization are: (1) it can model $\chi$-way variable interactions, where $\chi$ is the number of polynomial order and is usually set to two. (2) A fast optimization algorithm associated with factorization machines can reduce the polynomial computation time to linear complexity, making it extremely efficient especially for high dimensional sparse inputs.  For these reasons, factorization machines are widely employed in modern advertisement and products recommendations. The technical details and implementations are described below.


## 2-Way Factorization Machines

Formally, let $x \in \mathbb{R}^d$ denote the feature vectors of one sample, and $y$ denote the corresponding label which can be real-valued label or class label such as binary class "click/non-click". The model for a factorization machine of degree two is defined as:

$$
\hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j
$$

where $\mathbf{w}_0 \in \mathbb{R}$ is the global bias; $\mathbf{w} \in \mathbb{R}^d$ denotes the weights of the i-th variable; $\mathbf{V} \in \mathbb{R}^{d\times k}$ represents the feature embeddings; $\mathbf{v}_i$ represents the $i^\mathrm{th}$ row of $\mathbf{V}$; $k$ is the dimensionality of latent factors; $\langle\cdot, \cdot \rangle$ is the dot product of two vectors.  $\langle \mathbf{v}_i, \mathbf{v}_j \rangle$ model the interaction between the $i^\mathrm{th}$ and $j^\mathrm{th}$ feature. Some feature interactions can be easily understood so they can be designed by experts. However, most other feature interactions are hidden in data and difficult to identify. So modeling feature interactions automatically can greatly reduce the efforts in feature engineering. It is obvious that the first two terms correspond to the linear regression model and the last term is an extension of the matrix factorization model. If the feature $i$ represents an item and the feature $j$ represents a user, the third term is exactly the dot product between user and item embeddings. It is worth noting that FM can also generalize to higher orders (degree > 2). Nevertheless, the numerical stability might weaken the generalization.


## An Efficient Optimization Criterion

Optimizing the factorization machines in a  straight forward method leads to a complexity of $\mathcal{O}(kd^2)$ as all pairwise interactions require to be computed. To solve this inefficiency problem, we can reorganize the third term of FM which could greatly reduce the computation cost, leading to a linear time complexity ($\mathcal{O}(kd)$).  The reformulation of the pairwise interaction term is as follows:

$$
\begin{aligned}
&\sum_{i=1}^d \sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j \\
 &= \frac{1}{2} \sum_{i=1}^d \sum_{j=1}^d\langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j - \frac{1}{2}\sum_{i=1}^d \langle\mathbf{v}_i, \mathbf{v}_i\rangle x_i x_i \\
 &= \frac{1}{2} \big (\sum_{i=1}^d \sum_{j=1}^d \sum_{l=1}^k\mathbf{v}_{i, l} \mathbf{v}_{j, l} x_i x_j - \sum_{i=1}^d \sum_{l=1}^k \mathbf{v}_{i, l} \mathbf{v}_{i, l} x_i x_i \big)\\
 &=  \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i) (\sum_{j=1}^d \mathbf{v}_{j, l}x_j) - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2 \big ) \\
 &= \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)
 \end{aligned}
$$

With this reformulation, the model complexity are decreased greatly. Moreover, for sparse features, only non-zero elements needs to be computed so that the overall complexity is linear to the number of non-zero features.

To learn the FM model, we can use the MSE loss for regression task, the cross-entropy loss for classification tasks, and the BPR loss for ranking task. Standard optimizers such as stochastic gradient descent and Adam are viable for optimization.


In [ ]:
class FeaturesEmbedding(torch.nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)
    
class FactorizationMachine(torch.nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.linear = LogisticRegression(field_dims)
        self.feature_embedding = FeaturesEmbedding(field_dims, embed_dim)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        # zero+first order (linear)
        linear_predictions = self.linear(x)
        
        # second-order interaction
        embedding_feautre = self.feature_embedding(x)
        square_of_sum = torch.sum(embedding_feautre, dim=1) ** 2
        sum_of_square = torch.sum(embedding_feautre ** 2, dim=1)
        fm_predictions = square_of_sum - sum_of_square
        fm_predictions = 0.5 * torch.sum(fm_predictions, dim=1, keepdim=True).flatten()
        return fm_predictions + linear_predictions
        

In [ ]:
# training configs
EPOCHS = 10
lr = 1e-4
DIM = 16
device = "cuda"

# build model
model = FactorizationMachine(field_dims,embed_dim=DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
objective_function = nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(EPOCHS):
    for fields, target in train_loader:
        fields, target = fields.to(device), target.to(device)
        prediction = model(fields)
        loss = objective_function(prediction, target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    valid_roc = test(model, valid_loader,device)
    print(f"Validation ROC score:{valid_roc:.4f}")
    
test_roc = test(model, test_loader,device)
print(f"Testing ROC score:{test_roc:.4f}")

# From second-order to higher-order interaction

## Deep Factorization Machines

Learning effective feature combinations is critical to the success of click-through rate prediction task. Factorization machines model feature interactions in a linear paradigm (e.g., bilinear interactions). This is often insufficient for real-world data where inherent feature crossing structures are usually very complex and nonlinear. What's worse, second-order feature interactions are generally used in factorization machines in practice. Modeling higher degrees of feature combinations with factorization machines is possible theoretically but it is usually not adopted due to numerical instability and high computational complexity.

One effective solution is using deep neural networks. Deep neural networks are powerful in feature representation learning and have the potential to learn sophisticated feature interactions. As such, it is natural to integrate deep neural networks to factorization machines. Adding nonlinear transformation layers to factorization machines gives it the capability to model both low-order feature combinations and high-order feature combinations. Moreover, non-linear inherent structures from inputs can also be captured with deep neural networks. In this section, we will introduce a representative model named deep factorization machines (DeepFM) `Guo.Tang.Ye.ea.2017` which combine FM and deep neural networks.


## Model Architectures

DeepFM consists of an FM component and a deep component which are integrated in a parallel structure. The FM component is the same as the 2-way factorization machines which is used to model the low-order feature interactions. The deep component is an MLP that is used to capture high-order feature interactions and nonlinearities. These two components share the same inputs/embeddings and their outputs are summed up as the final prediction. It is worth pointing out that the spirit of DeepFM resembles that of the Wide \& Deep architecture which can capture both memorization and generalization. The advantages of DeepFM over the Wide \& Deep model is that it reduces the effort of hand-crafted feature engineering by identifying feature combinations automatically.

We omit the description of the FM component for brevity and denote the output as $\hat{y}^{(FM)}$. Readers are referred to the last section for more details. Let $\mathbf{e}_i \in \mathbb{R}^{k}$ denote the latent feature vector of the $i^\mathrm{th}$ field.  The input of the deep component is the concatenation of the dense embeddings of all fields that are looked up with the sparse categorical feature input, denoted as:

$$
\mathbf{z}^{(0)}  = [\mathbf{e}_1, \mathbf{e}_2, ..., \mathbf{e}_f],
$$

where $f$ is the number of fields.  It is then fed into the following neural network:

$$
\mathbf{z}^{(l)}  = \alpha(\mathbf{W}^{(l)}\mathbf{z}^{(l-1)} + \mathbf{b}^{(l)}),
$$

where $\alpha$ is the activation function.  $\mathbf{W}_{l}$ and $\mathbf{b}_{l}$ are the weight and bias at the $l^\mathrm{th}$ layer. Let $y_{DNN}$ denote the output of the prediction. The ultimate prediction of DeepFM is the summation of the outputs from both FM and DNN. So we have:

$$
\hat{y} = \sigma(\hat{y}^{(FM)} + \hat{y}^{(DNN)}),
$$

where $\sigma$ is the sigmoid function. The architecture of DeepFM is illustrated below.
![Illustration of the DeepFM model](https://raw.githubusercontent.com/d2l-ai/d2l-en-colab/master/img/rec-deepfm.svg)

It is worth noting that DeepFM is not the only way to combine deep neural networks with FM. We can also add nonlinear layers over the feature interactions `He.Chua.2017`.


In [ ]:
class MLP(torch.nn.Module):

    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, embed_dim)``
        """
        return self.mlp(x).flatten()

## Practice: implementing DeepFM
In this exercise, you need to implement the DeepFM model on your own.<br>
Specifically, DeepFM consists of two parts: FM and MLP.<br>
* FM: The same architecture as we described previously.
* MLP: The field embeddings are concatenated and passed into a MLP for learning higher-order interactions.


Based on that, please implement the DeepFM model accordingly. We provide a sketch of MLP module above.

In [ ]:
class DeepFM(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
        super().__init__()
        ############################################################################
        # TODO: Your code here!
        # define the necessary modules here

        ############################################################################

    def forward(self, x):
        """
        x: Long tensor of size ``(batch_size, num_fields)``
        """
        prediction = None
        ############################################################################
        # TODO: Your code here!
        # implement the forward pass here
    
        ############################################################################
        return prediction


In [ ]:
# training configs
EPOCHS = 10
lr = 1e-4
DIM = 16
MLP_DIMS = [DIM,DIM]
device = "cuda"

# build model
model = DeepFM(field_dims,embed_dim=DIM,mlp_dims=MLP_DIMS,dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
objective_function = nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(EPOCHS):
    for fields, target in train_loader:
        fields, target = fields.to(device), target.to(device)
        prediction = model(fields)
        loss = objective_function(prediction, target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    valid_roc = test(model, valid_loader,device)
    print(f"Validation ROC score:{valid_roc:.4f}")
    
test_roc = test(model, test_loader,device)
print(f"Testing ROC score:{test_roc:.4f}")

## Practice: Exploring other variants of FM family with `torchfm`
[Torchfm](https://github.com/rixwew/pytorch-fm) provides a PyTorch implementation of factorization machine models and common datasets in CTR prediction.<br>
Try to play with different models and see which one achieves the best performance!

## Available Models

| Model | Reference |
|-------|-----------|
| Logistic Regression | |
| Factorization Machine | [S Rendle, Factorization Machines, 2010.](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf) |
| Field-aware Factorization Machine | [Y Juan, et al. Field-aware Factorization Machines for CTR Prediction, 2015.](https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf) |
| Higher-Order Factorization Machines | [ M Blondel, et al. Higher-Order Factorization Machines, 2016.](https://dl.acm.org/doi/10.5555/3157382.3157473) |
| Factorization-Supported Neural Network | [W Zhang, et al. Deep Learning over Multi-field Categorical Data - A Case Study on User Response Prediction, 2016.](https://arxiv.org/abs/1601.02376) |
| Wide&Deep | [HT Cheng, et al. Wide & Deep Learning for Recommender Systems, 2016.](https://arxiv.org/abs/1606.07792) |
| Attentional Factorization Machine | [J Xiao, et al. Attentional Factorization Machines: Learning the Weight of Feature Interactions via Attention Networks, 2017.](https://arxiv.org/abs/1708.04617) |
| Neural Factorization Machine | [X He and TS Chua, Neural Factorization Machines for Sparse Predictive Analytics, 2017.](https://arxiv.org/abs/1708.05027) |
| Neural Collaborative Filtering | [X He, et al. Neural Collaborative Filtering, 2017.](https://arxiv.org/abs/1708.05031) |
| Field-aware Neural Factorization Machine | [L Zhang, et al. Field-aware Neural Factorization Machine for Click-Through Rate Prediction, 2019.](https://arxiv.org/abs/1902.09096) |
| Product Neural Network | [Y Qu, et al. Product-based Neural Networks for User Response Prediction, 2016.](https://arxiv.org/abs/1611.00144) |
| Deep Cross Network | [R Wang, et al. Deep & Cross Network for Ad Click Predictions, 2017.](https://arxiv.org/abs/1708.05123) |
| DeepFM | [H Guo, et al. DeepFM: A Factorization-Machine based Neural Network for CTR Prediction, 2017.](https://arxiv.org/abs/1703.04247) |
| xDeepFM | [J Lian, et al. xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems, 2018.](https://arxiv.org/abs/1803.05170) |
| AutoInt (Automatic Feature Interaction Model) | [W Song, et al. AutoInt: Automatic Feature Interaction Learning via Self-Attentive Neural Networks, 2018.](https://arxiv.org/abs/1810.11921) |
| AFN(AdaptiveFactorizationNetwork Model) | [Cheng W, et al. Adaptive Factorization Network: Learning Adaptive-Order Feature Interactions, AAAI'20.](https://arxiv.org/pdf/1909.03276.pdf) |

In [ ]:
! pip install torchfm

In [ ]:
from torchfm.model.afi import AutomaticFeatureInteractionModel
from torchfm.model.xdfm import ExtremeDeepFactorizationMachineModel

In [ ]:
# training configs
EPOCHS = 20
lr = 1e-4
DIM = 16
MLP_DIMS = [DIM,DIM]
device = "cuda"

# build model
# model = AutomaticFeatureInteractionModel(field_dims,embed_dim=DIM,atten_embed_dim=DIM,num_heads=2,num_layers=2,mlp_dims=MLP_DIMS,dropouts=[0,0]).to(device)
model = ExtremeDeepFactorizationMachineModel(field_dims,embed_dim=DIM,mlp_dims=MLP_DIMS,dropout=0.,cross_layer_sizes=MLP_DIMS).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
objective_function = nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(EPOCHS):
    for fields, target in train_loader:
        fields, target = fields.to(device), target.to(device)
        prediction = model(fields)
        loss = objective_function(prediction, target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    valid_roc = test(model, valid_loader,device)
    print(f"Validation ROC score:{valid_roc:.4f}")
    
test_roc = test(model, test_loader,device)
print(f"Testing ROC score:{test_roc:.4f}")